In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

train_df = pd.read_csv("/kaggle/input/titanic/train.csv")

test_df = pd.read_csv("/kaggle/input/titanic/test.csv")


In [18]:

# الاحتفاظ بـ PassengerId من بيانات الاختبار
test_passenger_ids = test_df["PassengerId"]

# -------------------------------
# 2. دمج البيانات مؤقتاً للتنظيف
# -------------------------------
full_df = pd.concat([train_df, test_df], sort=False)

# -------------------------------
# 3. معالجة القيم المفقودة
# -------------------------------
full_df["Age"].fillna(full_df["Age"].median(), inplace=True)
full_df["Fare"].fillna(full_df["Fare"].median(), inplace=True)
full_df["Embarked"].fillna(full_df["Embarked"].mode()[0], inplace=True)

# -------------------------------
# 4. تحويل الأعمدة النصية إلى رقمية
# -------------------------------
label_cols = ["Sex", "Embarked"]
for col in label_cols:
    le = LabelEncoder()
    full_df[col] = le.fit_transform(full_df[col])

# -------------------------------
# 5. استخراج ميزات إضافية
# -------------------------------
full_df["FamilySize"] = full_df["SibSp"] + full_df["Parch"] + 1
full_df["IsAlone"] = (full_df["FamilySize"] == 1).astype(int)
full_df["Age*Class"] = full_df["Age"] * full_df["Pclass"]
full_df["FarePerPerson"] = full_df["Fare"] / full_df["FamilySize"]

# -------------------------------
# 6. إسقاط الأعمدة غير المفيدة
# -------------------------------
drop_cols = ["PassengerId", "Name", "Ticket", "Cabin"]
full_df.drop(columns=drop_cols, inplace=True)

# -------------------------------
# 7. فصل البيانات مرة أخرى
# -------------------------------
train_clean = full_df.iloc[:len(train_df), :]
test_clean = full_df.iloc[len(train_df):, :]

X = train_clean.drop("Survived", axis=1)
y = train_clean["Survived"]


/tmp/ipykernel_36/1311114117.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_df["Age"].fillna(full_df["Age"].median(), inplace=True)
/tmp/ipykernel_36/1311114117.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

In [19]:

# 8. تقسيم البيانات لمجموعة تدريب والتحقق
# -------------------------------
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------------------
# 9. بناء نموذج XGBoost بأفضل المعلمات
# -------------------------------
best_xgb = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=1.0,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

# تدريب النموذج
best_xgb.fit(X_train, y_train)

# -------------------------------
# 10. اختبار النموذج على مجموعة التحقق
# -------------------------------
y_val_pred = best_xgb.predict(X_val)
print("Accuracy on validation set:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

# -------------------------------
# 11. التنبؤ على بيانات الاختبار
# -------------------------------
# إزالة أي عمود Survived إن وجد
X_test_final = test_clean.drop(columns=["Survived"], errors='ignore')
y_test_pred = best_xgb.predict(X_test_final)

# -------------------------------
# 12. إنشاء ملف submission
# -------------------------------
submission = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Survived': y_test_pred
})

submission.to_csv("titanic_submission_3.csv", index=False)
print("Submission file created successfully!")

Accuracy on validation set: 0.8491620111731844
              precision    recall  f1-score   support

         0.0       0.85      0.90      0.87       105
         1.0       0.84      0.78      0.81        74

    accuracy                           0.85       179
   macro avg       0.85      0.84      0.84       179
weighted avg       0.85      0.85      0.85       179

Submission file created successfully!
